In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Power Grid Analysis using Physics-Informed Neural Networks\n",
    "\n",
    "This notebook demonstrates the application of Physics-Informed Neural Networks (PINNs) for power grid analysis, including voltage stability prediction and power flow calculations.\n",
    "\n",
    "## Overview\n",
    "\n",
    "The notebook covers:\n",
    "1. Data generation and preprocessing\n",
    "2. Model initialization and architecture\n",
    "3. Training with physics-informed loss functions\n",
    "4. Model evaluation and visualization\n",
    "5. Power flow analysis and stability assessment"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import required libraries\n",
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from datetime import datetime\n",
    "import tensorflow as tf\n",
    "from tensorflow.keras import layers, optimizers\n",
    "\n",
    "# Set random seeds for reproducibility\n",
    "np.random.seed(42)\n",
    "tf.random.set_seed(42)\n",
    "\n",
    "# Import project-specific modules\n",
    "from src.models.power_grid_pinn import PowerGridPINN\n",
    "from src.utils.data_generator import PowerGridDataGenerator\n",
    "from src.config.config_manager import ConfigManager\n",
    "from src.utils.metrics import calculate_metrics\n",
    "from src.utils.visualization import plot_power_flow_results\n",
    "\n",
    "# Configure matplotlib\n",
    "plt.style.use('seaborn')\n",
    "plt.rcParams['figure.figsize'] = [12, 8]\n",
    "plt.rcParams['font.size'] = 12"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Generation and Preprocessing\n",
    "\n",
    "Generate synthetic power grid data for training and testing. The data includes:\n",
    "- Voltage magnitudes and angles\n",
    "- Active and reactive power injections\n",
    "- Line parameters and topology"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Initialize data generator with realistic parameters\n",
    "data_gen = PowerGridDataGenerator(\n",
    "    bus_count=5,          # Number of buses in the system\n",
    "    line_count=7,         # Number of transmission lines\n",
    "    base_mva=100,         # Base MVA for per-unit calculations\n",
    "    voltage_limits=(0.95, 1.05)  # Voltage limits in per-unit\n",
    ")\n",
    "\n",
    "# Generate training and test datasets\n",
    "train_data = data_gen.generate_dataset(n_samples=1000)\n",
    "test_data = data_gen.generate_dataset(n_samples=200)\n",
    "\n",
    "# Display dataset information\n",
    "print(\"Training Data Summary:\")\n",
    "for key, value in train_data.items():\n",
    "    if isinstance(value, np.ndarray):\n",
    "        print(f\"{key}: shape {value.shape}, range [{value.min():.3f}, {value.max():.3f}]\")\n",
    "\n",
    "print(\"\\nTest Data Summary:\")\n",
    "for key, value in test_data.items():\n",
    "    if isinstance(value, np.ndarray):\n",
    "        print(f\"{key}: shape {value.shape}, range [{value.min():.3f}, {value.max():.3f}]\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Model Initialization\n",
    "\n",
    "Initialize the Power Grid PINN model with appropriate architecture and hyperparameters."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load configuration\n",
    "config = ConfigManager().load_config('config/power_grid_config.yaml')\n",
    "\n",
    "# Initialize model with configuration parameters\n",
    "model = PowerGridPINN(\n",
    "    layers=config['model']['layers'],\n",
    "    learning_rate=config['training']['learning_rate'],\n",
    "    activation=config['model']['activation'],\n",
    "    dropout_rate=config['model']['dropout_rate']\n",
    ")\n",
    "\n",
    "# Display model architecture\n",
    "print(\"Model Architecture:\")\n",
    "model.summary()\n",
    "\n",
    "# Verify model parameters\n",
    "print(f\"\\nTotal trainable parameters: {model.count_params():,}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Model Training\n",
    "\n",
    "Train the model using both data and physics-based losses. The training process includes:\n",
    "- Data loss: MSE between predicted and actual values\n",
    "- Physics loss: Power flow equations and constraints\n",
    "- Regularization: L2 regularization for weights"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Training parameters\n",
    "epochs = config['training']['epochs']\n",
    "batch_size = config['training']['batch_size']\n",
    "validation_split = config['training']['validation_split']\n",
    "\n",
    "# Create TensorBoard callback for monitoring\n",
    "log_dir = f\"logs/power_grid_{datetime.now().strftime('%Y%m%d-%H%M%S')}\"\n",
    "tensorboard_callback = tf.keras.callbacks.TensorBoard(\n",
    "    log_dir=log_dir,\n",
    "    histogram_freq=1,\n",
    "    write_graph=True\n",
    ")\n",
    "\n",
    "# Train model\n",
    "history = model.train(\n",
    "    train_data,\n",
    "    epochs=epochs,\n",
    "    batch_size=batch_size,\n",
    "    validation_data=test_data,\n",
    "    callbacks=[tensorboard_callback]\n",
    ")\n",
    "\n",
    "# Save training history\n",
    "history_df = pd.DataFrame(history)\n",
    "history_df.to_csv('results/power_grid_training_history.csv', index=False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Training Visualization\n",
    "\n",
    "Visualize the training process and loss components to monitor model convergence."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def plot_training_history(history):\n",
    "    \"\"\"Plot training history with multiple loss components.\"\"\"\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "    \n",
    "    # Total loss\n",
    "    axes[0, 0].plot(history['total_loss'], label='Training')\n",
    "    axes[0, 0].plot(history['val_total_loss'], label='Validation')\n",
    "    axes[0, 0].set_title('Total Loss')\n",
    "    axes[0, 0].set_xlabel('Epoch')\n",
    "    axes[0, 0].set_ylabel('Loss')\n",
    "    axes[0, 0].legend()\n",
    "    \n",
    "    # Physics loss\n",
    "    axes[0, 1].plot(history['physics_loss'], label='Training')\n",
    "    axes[0, 1].plot(history['val_physics_loss'], label='Validation')\n",
    "    axes[0, 1].set_title('Physics Loss')\n",
    "    axes[0, 1].set_xlabel('Epoch')\n",
    "    axes[0, 1].set_ylabel('Loss')\n",
    "    axes[0, 1].legend()\n",
    "    \n",
    "    # Data loss\n",
    "    axes[1, 0].plot(history['data_loss'], label='Training')\n",
    "    axes[1, 0].plot(history['val_data_loss'], label='Validation')\n",
    "    axes[1, 0].set_title('Data Loss')\n",
    "    axes[1, 0].set_xlabel('Epoch')\n",
    "    axes[1, 0].set_ylabel('Loss')\n",
    "    axes[1, 0].legend()\n",
    "    \n",
    "    # Learning rate\n",
    "    axes[1, 1].plot(history['lr'], label='Learning Rate')\n",
    "    axes[1, 1].set_title('Learning Rate')\n",
    "    axes[1, 1].set_xlabel('Epoch')\n",
    "    axes[1, 1].set_ylabel('Learning Rate')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.savefig('results/power_grid_training_history.png')\n",
    "    plt.show()\n",
    "\n",
    "# Plot training history\n",
    "plot_training_history(history)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Model Evaluation\n",
    "\n",
    "Evaluate the model's performance on the test dataset using various metrics."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Make predictions\n",
    "predictions = model.predict(test_data)\n",
    "\n",
    "# Calculate metrics\n",
    "metrics = calculate_metrics(test_data, predictions)\n",
    "\n",
    "# Display metrics\n",
    "print(\"Model Performance Metrics:\")\n",
    "for metric, value in metrics.items():\n",
    "    print(f\"{metric}: {value:.4f}\")\n",
    "\n",
    "# Save metrics to file\n",
    "pd.DataFrame([metrics]).to_csv('results/power_grid_metrics.csv', index=False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Visualization of Results\n",
    "\n",
    "Visualize the model's predictions and compare them with actual values."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def plot_predictions_vs_actual(test_data, predictions):\n",
    "    \"\"\"Plot predictions against actual values for voltage and power.\"\"\"\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(15, 12))\n",
    "    \n",
    "    # Voltage magnitude\n",
    "    axes[0, 0].scatter(test_data['voltage_magnitude'], predictions['voltage_magnitude'], alpha=0.5)\n",
    "    axes[0, 0].plot([0.9, 1.1], [0.9, 1.1], 'r--')\n",
    "    axes[0, 0].set_xlabel('Actual Voltage Magnitude')\n",
    "    axes[0, 0].set_ylabel('Predicted Voltage Magnitude')\n",
    "    axes[0, 0].set_title('Voltage Magnitude Prediction')\n",
    "    \n",
    "    # Voltage angle\n",
    "    axes[0, 1].scatter(test_data['voltage_angle'], predictions['voltage_angle'], alpha=0.5)\n",
    "    axes[0, 1].plot([-np.pi, np.pi], [-np.pi, np.pi], 'r--')\n",
    "    axes[0, 1].set_xlabel('Actual Voltage Angle')\n",
    "    axes[0, 1].set_ylabel('Predicted Voltage Angle')\n",
    "    axes[0, 1].set_title('Voltage Angle Prediction')\n",
    "    \n",
    "    # Active power\n",
    "    axes[1, 0].scatter(test_data['active_power'], predictions['active_power'], alpha=0.5)\n",
    "    axes[1, 0].plot([-1, 1], [-1, 1], 'r--')\n",
    "    axes[1, 0].set_xlabel('Actual Active Power')\n",
    "    axes[1, 0].set_ylabel('Predicted Active Power')\n",
    "    axes[1, 0].set_title('Active Power Prediction')\n",
    "    \n",
    "    # Reactive power\n",
    "    axes[1, 1].scatter(test_data['reactive_power'], predictions['reactive_power'], alpha=0.5)\n",
    "    axes[1, 1].plot([-1, 1], [-1, 1], 'r--')\n",
    "    axes[1, 1].set_xlabel('Actual Reactive Power')\n",
    "    axes[1, 1].set_ylabel('Predicted Reactive Power')\n",
    "    axes[1, 1].set_title('Reactive Power Prediction')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.savefig('results/power_grid_predictions.png')\n",
    "    plt.show()\n",
    "\n",
    "# Plot predictions\n",
    "plot_predictions_vs_actual(test_data, predictions)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Power Flow Analysis\n",
    "\n",
    "Analyze the power flow results and assess system stability."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Perform power flow analysis\n",
    "flow_results = model.analyze_power_flow(test_data)\n",
    "\n",
    "# Plot power flow results\n",
    "plot_power_flow_results(flow_results)\n",
    "\n",
    "# Save results\n",
    "pd.DataFrame(flow_results).to_csv('results/power_flow_analysis.csv', index=False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Save Model and Results\n",
    "\n",
    "Save the trained model and all results for future use."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Save model\n",
    "model.save('models/power_grid_model.h5')\n",
    "print(\"Model saved successfully!\")\n",
    "\n",
    "# Save configuration\n",
    "config.save('results/power_grid_config.yaml')\n",
    "print(\"Configuration saved successfully!\")\n",
    "\n",
    "# Print summary\n",
    "print(\"\\nTraining Summary:\")\n",
    "print(f\"Total epochs: {epochs}\")\n",
    "print(f\"Final total loss: {history['total_loss'][-1]:.4f}\")\n",
    "print(f\"Final physics loss: {history['physics_loss'][-1]:.4f}\")\n",
    "print(f\"Final data loss: {history['data_loss'][-1]:.4f}\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined